In [16]:
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from osgeo import gdal, ogr

In [17]:
# Load NDVI raster image
ndvi_file = 'C:/Uni/4_Semester/Earth_Observation/Final Project/daten/NDVI_CIR_reproj.tif'
ndvi_dataset = gdal.Open(ndvi_file)
ndvi_array = ndvi_dataset.ReadAsArray()

In [18]:
# Load control points from the shapefile
shapefile = 'C:/Uni/4_Semester/Earth_Observation/Final Project/daten/control_point_fusion.shp'
driver = ogr.GetDriverByName('ESRI Shapefile')
dataset = driver.Open(shapefile)
layer = dataset.GetLayer()

shapefile

'C:/Uni/4_Semester/Earth_Observation/Final Project/daten/control_point_fusion.shp'

In [20]:
control_points = []

for feature in layer:
    geometry = feature.GetGeometryRef()
    x = geometry.GetX()
    y = geometry.GetY()
    label = feature.GetField('Class_ID')  
    ndvi_value = feature.GetField('ndvi_cir')  # variable nueva
    control_points.append([ndvi_value, label])

In [5]:
#control_points = []   #new data


#for feature in layer:
#    geometry = feature.GetGeometryRef()
#    x = geometry.GetX()
#    y = geometry.GetY()
#    label = feature.GetField('Class_ID')  
#    high = feature.GetField('high') # new variable
#    ndvi_value = feature.GetField('ndvi_cir')  
 #   control_points.append([ndvi_value, label, high])

In [21]:
# Convert the control points to a numpy arra
control_points = np.array(control_points)
control_points

array([[0.18868, 2.     ],
       [0.02738, 1.     ],
       [0.23694, 2.     ],
       [0.18733, 2.     ],
       [0.22843, 2.     ],
       [0.22823, 2.     ],
       [0.06201, 1.     ],
       [0.11626, 1.     ],
       [0.01486, 1.     ],
       [0.10849, 1.     ],
       [0.09311, 1.     ],
       [0.23913, 2.     ],
       [0.14168, 2.     ],
       [0.16356, 2.     ],
       [0.23796, 2.     ],
       [0.15691, 1.     ],
       [0.14836, 1.     ],
       [0.269  , 1.     ],
       [0.13606, 1.     ],
       [0.20474, 1.     ],
       [0.17715, 2.     ],
       [0.15851, 2.     ],
       [0.13498, 1.     ],
       [0.13995, 1.     ],
       [0.27296, 2.     ],
       [0.12122, 1.     ],
       [0.1633 , 1.     ],
       [0.13212, 1.     ],
       [0.08084, 1.     ],
       [0.21871, 2.     ],
       [0.16052, 2.     ],
       [0.23132, 2.     ],
       [0.20154, 2.     ],
       [0.16101, 1.     ],
       [0.1469 , 1.     ],
       [0.12833, 1.     ],
       [0.1483 , 1.     ],
 

In [22]:
#control_points[:,2]

IndexError: index 2 is out of bounds for axis 1 with size 2

In [11]:
# Obtener datos de altura de los árboles
#height_values = control_points[:, 2].reshape(-1, 1)  # Ajusta la columna correspondiente a los datos de altura en tu control_points
#height_values

array([[44.36],
       [39.36],
       [43.7],
       [44.76],
       [43.5],
       [42.85],
       [38.67],
       [45.28],
       [38.69],
       [40.51],
       [45.37],
       [31.4],
       [31.4],
       [31.4],
       [37.99],
       [43.83],
       [31.4],
       [None],
       [None],
       [31.4],
       [41.77],
       [40.44],
       [42.71],
       [55.81],
       [44.12],
       [50.64],
       [56.9],
       [53.25],
       [43.86],
       [40.79],
       [39.67],
       [45.21],
       [31.4],
       [31.4],
       [31.4],
       [None],
       [31.4],
       [37.75],
       [38.04],
       [31.4],
       [31.4],
       [42.7],
       [37.22],
       [37.13],
       [40.61],
       [50.87],
       [48.95],
       [36.09],
       [50.38]], dtype=object)

In [23]:
# Obtain NDVI values and control point labels
ndvi_values = control_points[:, 0].reshape(-1, 1)
labels = control_points[:, 1] #X
ndvi_values #y

array([[0.18868],
       [0.02738],
       [0.23694],
       [0.18733],
       [0.22843],
       [0.22823],
       [0.06201],
       [0.11626],
       [0.01486],
       [0.10849],
       [0.09311],
       [0.23913],
       [0.14168],
       [0.16356],
       [0.23796],
       [0.15691],
       [0.14836],
       [0.269  ],
       [0.13606],
       [0.20474],
       [0.17715],
       [0.15851],
       [0.13498],
       [0.13995],
       [0.27296],
       [0.12122],
       [0.1633 ],
       [0.13212],
       [0.08084],
       [0.21871],
       [0.16052],
       [0.23132],
       [0.20154],
       [0.16101],
       [0.1469 ],
       [0.12833],
       [0.1483 ],
       [0.137  ],
       [0.25333],
       [0.26251],
       [0.1824 ],
       [0.22672],
       [0.1481 ],
       [0.16624],
       [0.1325 ],
       [0.10854],
       [0.07696],
       [0.04421],
       [0.14327]])

In [24]:
labels

array([2., 1., 2., 2., 2., 2., 1., 1., 1., 1., 1., 2., 2., 2., 2., 1., 1.,
       1., 1., 1., 2., 2., 1., 1., 2., 1., 1., 1., 1., 2., 2., 2., 2., 1.,
       1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1.])

In [25]:
# Training a classification model
# Change 'SVM' to 'RandomForestClassifier' if you want to use a random forest classifier.
#classifier = svm.SVC()  # Support Vector Machine (SVM) classifier
classifier = RandomForestClassifier()  # Random Forest Classifier
classifier.fit(ndvi_values, labels) ##clf

RandomForestClassifier()

In [50]:
# Classify the complete raster image
#classified_array = np.zeros_like(ndvi_array)  # Matrix for storing classified results

#rows, cols = ndvi_array.shape
#for row in range(rows):
#    for col in range(cols):
#        ndvi_value = ndvi_array[row, col]
#        predicted_label = classifier.predict([[ndvi_value]])[0]
#        classified_array[row, col] = predicted_label

In [26]:
# Classify the complete raster image
classified_array = np.zeros_like(ndvi_array)  # Matrix for storing classified results

rows, cols = ndvi_array.shape
for row in range(rows):
    for col in range(cols):
        ndvi_value = ndvi_array[row, col]
        if ndvi_value >= 0:
            predicted_label = classifier.predict([[ndvi_value]])[0]
            classified_array[row, col] = predicted_label
        else:
            classified_array[row, col] = 0  # Asignar no vegetación a los valores NDVI negativos


In [61]:
# Save the sorted image
#classified_image = 'C:/Uni/4_Semester/Earth_Observation/Final Project/daten/ndvi_version1.tif'
#driver = gdal.GetDriverByName('GTiff')
#output_dataset = driver.Create(classified_image, cols, rows, 1, gdal.GDT_Float32)
#output_dataset.GetRasterBand(1).WriteArray(classified_array)
#output_dataset.FlushCache()

In [27]:
# Save the classified file with the appropriate projection
classified_image = 'C:/Uni/4_Semester/Earth_Observation/Final Project/daten/ndvi_clas4.tif'
driver = gdal.GetDriverByName('GTiff')
output_dataset = driver.Create(classified_image, cols, rows, 1, gdal.GDT_Float32)
output_dataset.GetRasterBand(1).WriteArray(classified_array)


0

In [28]:
# Obtain the projection of the original NDVI raster
ndvi_projection = ndvi_dataset.GetProjection()
output_dataset.SetProjection(ndvi_projection)

0

In [29]:
# Obtain the geospatial transformation of the original NDVI raster
ndvi_transform = ndvi_dataset.GetGeoTransform()
output_dataset.SetGeoTransform(ndvi_transform)

0

In [30]:
# Save changes and close the datasets
output_dataset.FlushCache()
output_dataset = None
ndvi_dataset = None

print("Classification completed. Sorted image saved in", classified_image)

Classification completed. Sorted image saved in C:/Uni/4_Semester/Earth_Observation/Final Project/daten/ndvi_clas4.tif


In [62]:
# Close the datasets
#ndvi_dataset = None
#output_dataset = None
#dataset = None

#print("Classification completed. Sorted image saved in", classified_image)


Classification completed. Sorted image saved in C:/Uni/4_Semester/Earth_Observation/Final Project/daten/ndvi_version1.tif


# Pixels > 0


In [28]:
# Cargar la imagen raster NDVI
ndvi_file = 'C:/Uni/4_Semester/Earth_Observation/Final Project/daten/NDVI_CIR_reproj.tif'
ndvi_dataset = gdal.Open(ndvi_file)
ndvi_band = ndvi_dataset.GetRasterBand(1)
ndvi_array = ndvi_band.ReadAsArray()


In [29]:
# Cargar los puntos de control desde el shapefile
shapefile = 'C:/Uni/4_Semester/Earth_Observation/Final Project/daten/control_point_fusion.shp'
driver = ogr.GetDriverByName('ESRI Shapefile')
dataset = driver.Open(shapefile)
layer = dataset.GetLayer()

In [30]:
control_points = []

for feature in layer:
    geometry = feature.GetGeometryRef()
    x = geometry.GetX()
    y = geometry.GetY()
    label = feature.GetField('Class_ID')  # Reemplaza 'etiqueta' con el nombre del campo que contiene las etiquetas
    ndvi_value = feature.GetField('ndvi_cir')  # Reemplaza 'ndvi' con el nombre del campo que contiene los valores de NDVI
    control_points.append([ndvi_value, label])

In [31]:
# Convertir los puntos de control en un array numpy
control_points = np.array(control_points)

In [32]:
# Obtener los valores de NDVI y las etiquetas de los puntos de control
ndvi_values = control_points[:, 0].reshape(-1, 1)
labels = control_points[:, 1]


In [35]:
# Dividir los puntos de control en conjuntos de entrenamiento y prueba
train_ndvi, test_ndvi, train_labels, test_labels = train_test_split(ndvi_values, labels, test_size=0.2, random_state=42)


In [36]:
# Entrenar un modelo de clasificación
# Cambia 'SVM' por 'RandomForestClassifier' si deseas utilizar un clasificador de bosque aleatorio
classifier = svm.SVC()  # Clasificador de Máquinas de Vectores de Soporte (SVM)
# classifier = RandomForestClassifier()  # Clasificador de Bosque Aleatorio
classifier.fit(train_ndvi, train_labels)


SVC()

In [37]:
# Clasificar la imagen raster completa
rows, cols = ndvi_array.shape
classified_array = np.zeros((rows, cols), dtype=np.uint8)  # Matriz para almacenar los resultados clasificados

for row in range(rows):
    for col in range(cols):
        ndvi_value = ndvi_array[row, col]
        if ndvi_value > 0:
            predicted_label = classifier.predict([[ndvi_value]])[0]
            if predicted_label == 'caducifolio':
                classified_array[row, col] = 1


In [40]:
# Calcular la precisión del modelo utilizando los conjuntos de prueba
test_predictions = classifier.predict(test_ndvi)
accuracy = accuracy_score(test_labels, test_predictions)

In [41]:
# Guardar la imagen clasificada
classified_image = 'C:/Uni/4_Semester/Earth_Observation/Final Project/daten/mayoracero.tif'
driver = gdal.GetDriverByName('GTiff')
output_dataset = driver.Create(classified_image, cols, rows, 1, gdal.GDT_Byte)
output_dataset.GetRasterBand(1).WriteArray(classified_array)
output_dataset.GetRasterBand(1).SetNoDataValue(0)  # Píxeles sin clasificar se establecen como 0
output_dataset.FlushCache()

In [42]:
# Cerrar los datasets
ndvi_dataset = None
output_dataset = None
dataset = None

print("Clasificación completada. Imagen clasificada guardada en", classified_image)
print("Precisión del modelo:", accuracy)


Clasificación completada. Imagen clasificada guardada en C:/Uni/4_Semester/Earth_Observation/Final Project/daten/mayoracero.tif
Precisión del modelo: 0.5


In [16]:
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from osgeo import gdal, ogr

# Cargar la imagen raster NDVI
ndvi_file = 'C:/Uni/4_Semester/Earth_Observation/Final Project/daten/NDVI_CIR_reproj.tif'
ndvi_dataset = gdal.Open(ndvi_file)
ndvi_array = ndvi_dataset.ReadAsArray()

# Cargar los puntos de control desde el shapefile
shapefile = 'C:/Uni/4_Semester/Earth_Observation/Final Project/daten/control_point_fusion.shp'
driver = ogr.GetDriverByName('ESRI Shapefile')
dataset = driver.Open(shapefile)
layer = dataset.GetLayer()
control_points = []

for feature in layer:
    geometry = feature.GetGeometryRef()
    x = geometry.GetX()
    y = geometry.GetY()
    label = feature.GetField('Class_ID')  # Reemplaza 'etiqueta' con el nombre del campo que contiene las etiquetas
    ndvi_value = feature.GetField('ndvi_cir')  # Reemplaza 'ndvi' con el nombre del campo que contiene los valores de NDVI
    control_points.append([ndvi_value, label])

# Convertir los puntos de control en un array numpy
control_points = np.array(control_points)

# Obtener los valores de NDVI y las etiquetas de los puntos de control
ndvi_values = control_points[:, 0].reshape(-1, 1)
labels = control_points[:, 1]

# Entrenar un modelo de clasificación
# Cambia 'SVM' por 'RandomForestClassifier' si deseas utilizar un clasificador de bosque aleatorio
classifier = svm.SVC()  # Clasificador de Máquinas de Vectores de Soporte (SVM)
# classifier = RandomForestClassifier()  # Clasificador de Bosque Aleatorio
classifier.fit(ndvi_values, labels)

# Clasificar la imagen raster completa
rows, cols = ndvi_array.shape
classified_array = np.zeros_like(ndvi_array, dtype=np.uint8)  # Matriz para almacenar los resultados clasificados

for row in range(rows):
    for col in range(cols):
        ndvi_value = ndvi_array[row, col]
        predicted_label = classifier.predict([[ndvi_value]])[0]
        if predicted_label == 'caducifolio':
            classified_array[row, col] = 1
        else:
            classified_array[row, col] = 0

# Establecer los valores no clasificados como nodata
classified_array[classified_array == 0] = ndvi_dataset.GetRasterBand(1).GetNoDataValue()

# Guardar la imagen clasificada
classified_image = 'C:/Uni/4_Semester/Earth_Observation/Final Project/daten/otro_intento.tif'
driver = gdal.GetDriverByName('GTiff')
output_dataset = driver.Create(classified_image, cols, rows, 1, gdal.GDT_Byte)
output_dataset.GetRasterBand(1).WriteArray(classified_array)
output_dataset.FlushCache()
output_dataset.GetRasterBand(1).SetNoDataValue(ndvi_dataset.GetRasterBand(1).GetNoDataValue())

# Cerrar los datasets
ndvi_dataset = None
output_dataset = None
dataset = None

print("Clasificación completada. Imagen clasificada guardada en", classified_image)


OverflowError: Python int too large to convert to C long